# Analysis of Australian Political Ads on Facebook
Author: Joshua Wang

In [1]:
from pyspark.sql.functions import concat, col, lower, lit, when, split

## Dataset Exploration and Cleaning

### Load datasets being used

In [2]:
# Load political ad dataset
df = spark.read.option("header", True).json('/data/ProjectDatasetFacebookAU')

In [3]:
# Load all candidates and successfully elected candidates for 2022 federal election (for both house of representatives and senate)
house_candidates = spark.read.option("header", True).csv('/user/s4696561/project/house-candidates.csv')
house_elected = spark.read.option("header", True).csv('/user/s4696561/project/representatives.csv')
senate_candidates = spark.read.option("header", True).csv('/user/s4696561/project/senate-candidates.csv')
senate_elected = spark.read.option("header", True).csv('/user/s4696561/project/senators.csv')

### Schemas of datasets

In [4]:
df.printSchema()

root
 |-- ad_creation_time: string (nullable = true)
 |-- ad_creative_bodies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ad_creative_body: string (nullable = true)
 |-- ad_creative_link_caption: string (nullable = true)
 |-- ad_creative_link_captions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ad_creative_link_description: string (nullable = true)
 |-- ad_creative_link_descriptions: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ad_creative_link_title: string (nullable = true)
 |-- ad_creative_link_titles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ad_delivery_start_time: string (nullable = true)
 |-- ad_delivery_stop_time: string (nullable = true)
 |-- ad_snapshot_url: string (nullable = true)
 |-- bylines: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- delivery_by_region: array (nullable = true)
 |    |-- element: struct (containsN

In [5]:
df.head()

24/04/25 03:13:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Row(ad_creation_time='2021-11-21', ad_creative_bodies=None, ad_creative_body="I'm ready to fight Scott Morrison's Cashless Pension Card being rolled out in Lilley.", ad_creative_link_caption=None, ad_creative_link_captions=None, ad_creative_link_description=None, ad_creative_link_descriptions=None, ad_creative_link_title='Cashless Pension Card', ad_creative_link_titles=None, ad_delivery_start_time='2021-11-21', ad_delivery_stop_time=None, ad_snapshot_url='https://www.facebook.com/ads/archive/render_ad/?id=966461410889234&access_token=EAAG7mM7GdIsBADfD3puwsjZBZCoYLv4bKLXwzw6Fdne1DYSDXo0je6isRr8yEgaZAxFncyzRs1RPJxebw77MaXZBI12hOs9cjPVAN11L4vrY6ddT1dEZCoEOFje4uBkA1Nn6kY3FkgO7zZBNpxUpkq2ijrzRaa2Mm1EyZCGdz3KcS6KDaHduw6NfcorBcQwKq0ZD', bylines=None, currency='AUD', delivery_by_region=None, demographic_distribution=[Row(age='55-64', gender='unknown', percentage='0.001232'), Row(age='65+', gender='unknown', percentage='0.017252'), Row(age='55-64', gender='female', percentage='0.057917'), Row(a

In [6]:
house_candidates.printSchema()
house_elected.printSchema()
senate_candidates.printSchema()
senate_elected.printSchema()

root
 |-- state: string (nullable = true)
 |-- division: string (nullable = true)
 |-- ballotPosition: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- ballotGivenName: string (nullable = true)
 |-- partyBallotName: string (nullable = true)

root
 |-- Person ID: string (nullable = true)
 |-- First name: string (nullable = true)
 |-- Last name: string (nullable = true)
 |-- Party: string (nullable = true)
 |-- Division: string (nullable = true)
 |-- URI: string (nullable = true)

root
 |-- state: string (nullable = true)
 |-- column: string (nullable = true)
 |-- ballotPosition: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- ballotGivenName: string (nullable = true)
 |-- groupName: string (nullable = true)
 |-- partyBallotName: string (nullable = true)

root
 |-- Person ID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Party: string (nullable = true)
 |-- URI: string (nullable = true)



### Cleaning Data

In [7]:
ad_data = df.select(
    "id",
    "page_id",
    "demographic_distribution", # breakdown of age and gender groups presented with the ad
    "funding_entity", # person paying for ad
    "page_name", # page paying for ad (likely the same as funding_entity, included for checking)
    "impressions", # number of people who saw the ad - gives a lower and upper bound
    "region_distribution", # which states/territories the ad is run in
    "spend", # cost of ad - gives a lower and upper bound
    "ad_creation_time",
    "ad_delivery_start_time",
    "ad_delivery_stop_time"
)

In [8]:
# Need to fix names since candidates file includes middle names but elected people only records first names - this will mess up joins and ad searching so change to only using first names
house_candidates = house_candidates.withColumn("ballotGivenName", split(col("ballotGivenName"), " ")[0])
senate_candidates = senate_candidates.withColumn("ballotGivenName", split(col("ballotGivenName"), " ")[0])

In [9]:
# In 2023, Cameron Caldwell was elected after the resignation of Stuart Robert.
# Since my analysis is focussed on ad spending leading up to the 2022 federal elections, Stuart Robert will be considered as "elected" rather than Cameron.
# Manually replace Cameron Caldwell with Stuart Robert in successful candidates dataframe.
house_elected = house_elected.withColumn(
    "First name",
    when(house_elected["First name"] == "Cameron", "Stuart")
    .otherwise(house_elected["First name"])
).withColumn(
    "Last name",
    when(house_elected["Last name"] == "Caldwell", "Robert")
    .otherwise(house_elected["Last name"])
)

In [10]:
# Get full names for all entries in each dataset and convert all to lowercase for easier comparison with ad data
house_candidate_concatenated = house_candidates.withColumn("full_name", concat(lower(col("ballotGivenName")), lit(" "), lower(col("surname"))))
senate_candidate_concatenated = senate_candidates.withColumn("full_name", concat(lower(col("ballotGivenName")), lit(" "), lower(col("surname"))))
house_elected_concatenated = house_elected.withColumn("full_name", concat(lower(col("First name")), lit(" "), lower(col("Last name"))))
senate_elected_concatenated = senate_elected.withColumn("full_name", lower(col("Name"))) 

In [11]:
# Only take data we need for more efficient processing and rename data for consistency
house_candidates_details = house_candidate_concatenated.select("full_name", "partyBallotName", "state").withColumnRenamed("partyBallotName", "party")
senate_candidates_details = senate_candidate_concatenated.select("full_name", "partyBallotName", "state").withColumnRenamed("partyBallotName", "party")

In [23]:
# Some parties used different names on ballots - standardise these
house_candidates_details = house_candidates_details.withColumn(
    "party",
    when(
        (house_candidates_details["party"] == "A.L.P.") | (house_candidates_details["party"] == "Labor"),
        "Australian Labor Party"
    ).when(
        (house_candidates_details["party"] == "The Greens (WA)") | (house_candidates_details["party"] == "Queensland Greens")
        | (house_candidates_details["party"] == "The Greens"),
        "Australian Greens"
    ).when(
        (house_candidates_details["party"] == "Liberal National Party of Queensland") | (house_candidates_details["party"] == "Liberal")
        | (house_candidates_details["party"] == "Liberal Party") | (house_candidates_details["party"] == "National Party")
        | (house_candidates_details["party"] == "The Nationals"),
        "Liberal-National Coalition" # Consider liberal and national parties as one party since they often run together
    ).when(
        (house_candidates_details["party"] == "Democratic Alliance"),
        "Drew Pavlou Democratic Alliance"
    ).when(
        (house_candidates_details["party"].isNull()),
        "Independent"
    )
    .otherwise(house_candidates_details["party"])
)

senate_candidates_details = senate_candidates_details.withColumn(
    "party",
    when(
        (senate_candidates_details["party"] == "A.L.P.") | (senate_candidates_details["party"] == "Labor"),
        "Australian Labor Party"
    ).when(
        (senate_candidates_details["party"] == "The Greens (WA)") | (senate_candidates_details["party"] == "Queensland Greens")
        | (senate_candidates_details["party"] == "The Greens"),
        "Australian Greens"
    ).when(
        (senate_candidates_details["party"] == "Liberal National Party of Queensland") | (senate_candidates_details["party"] == "Liberal")
        | (senate_candidates_details["party"] == "Liberal Party") | (senate_candidates_details["party"] == "National Party")
        | (senate_candidates_details["party"] == "The Nationals"),
        "Liberal-National Coalition" # Consider liberal and national parties as one party since they often run together
    ).when(
        (senate_candidates_details["party"] == "Democratic Alliance"),
        "Drew Pavlou Democratic Alliance"
    ).when(
        (senate_candidates_details["party"].isNull()),
        "Independent"
    )
    .otherwise(senate_candidates_details["party"])
)

In [24]:
# Dataset of elected candidates doesn't contain state represented - get this from all candidates dataset
house_elected_details = house_elected_concatenated.join(
    house_candidates_details,
    house_candidates_details.full_name == house_elected_concatenated.full_name,
    'left'
).select(
    house_elected_concatenated.full_name,
    house_candidates_details.party,
    "state"
)

# There are 2 "David Smiths" and no way to filter based off information available, so drop the incorrect one manually
house_elected_details = house_elected_details.filter(
    ~((house_elected_details["full_name"] == "david smith")
      & (house_elected_details["party"] == "United Australia Party"))
)

In [25]:
# Only 40 of the total 76 seats in the senate were availabe during the 2022 federal election (was a half senate election, other half continuing from 2019).
# We will only analyse ad spending of senate candidates running in the 2022 election, as ad data for 2019 is unavailable.
# Resignations and other factors result in only 36 senators available for analysis.
senate_elected_details = senate_elected_concatenated.join(
    senate_candidates_details,
    senate_candidates_details.full_name == senate_elected_concatenated.full_name,
    'inner'
).select(
    senate_elected_concatenated.full_name,
    senate_candidates_details.party,
    "state"
)

### Resulting Schemas and Sizes of Datasets

In [12]:
ad_data.printSchema()

root
 |-- id: string (nullable = true)
 |-- page_id: string (nullable = true)
 |-- demographic_distribution: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- age: string (nullable = true)
 |    |    |-- gender: string (nullable = true)
 |    |    |-- percentage: string (nullable = true)
 |-- funding_entity: string (nullable = true)
 |-- page_name: string (nullable = true)
 |-- impressions: struct (nullable = true)
 |    |-- lower_bound: string (nullable = true)
 |    |-- upper_bound: string (nullable = true)
 |-- region_distribution: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- percentage: string (nullable = true)
 |    |    |-- region: string (nullable = true)
 |-- spend: struct (nullable = true)
 |    |-- lower_bound: string (nullable = true)
 |    |-- upper_bound: string (nullable = true)
 |-- ad_creation_time: string (nullable = true)
 |-- ad_delivery_start_time: string (nullable = true)
 |-- ad_deliver

In [13]:
house_candidates_details.printSchema()
senate_candidates_details.printSchema()
house_elected_details.printSchema()
senate_elected_details.printSchema()

root
 |-- full_name: string (nullable = true)
 |-- party: string (nullable = true)
 |-- state: string (nullable = true)

root
 |-- full_name: string (nullable = true)
 |-- party: string (nullable = true)
 |-- state: string (nullable = true)

root
 |-- full_name: string (nullable = true)
 |-- party: string (nullable = true)
 |-- state: string (nullable = true)

root
 |-- full_name: string (nullable = true)
 |-- party: string (nullable = true)
 |-- state: string (nullable = true)

